In [2]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import json
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
CheckPageInfo = True
CheckPageNum = 1
Page = []
while CheckPageInfo:
    req1 = requests.get(f'https://www.huashan1914.com/w/huashan1914/exhibition?index={CheckPageNum}')
    soup1 = bs(req1.text, 'html5lib')
    findAllList = soup1.find('ul', {'id':'event-ul'}).findAll('li')
    if findAllList:
        for i in findAllList:
            activityName = i.find("div", {"class":"card-text-name"}).text
            activityDate = i.find("div", {"class":"event-date"}).text.strip()
            activityDateCut = activityDate.split(' - ')
            if len(activityDateCut) == 2:
                if len(activityDateCut[1]) > 6:
                    # 跨年分活動
                    activityDateStart, activityDateEnd = activityDateCut[0], activityDateCut[1]
                else:
                    # 當一年活動
                    activityDateStart, activityDateEnd = activityDateCut[0], activityDateCut[0][0:5] + activityDateCut[1]
            else:
                # 當天活動
                activityDateStart, activityDateEnd = activityDateCut[0], activityDateCut[0]

            activityUrl = 'https://www.huashan1914.com' + i.find("a")['href']
            activityType = [c.text for c in i.find("div", {'class':'event-list-type'}).findAll('span')]
            Page.append((activityName, activityDateStart, activityDateEnd, activityUrl, activityType))
        CheckPageNum+=1
#         break
    else:
        CheckPageInfo = False

In [4]:
DF = pd.DataFrame(Page, columns=['title', 'start', 'end', 'url', 'description'])

In [5]:
DF['start'] = pd.to_datetime(DF['start'], format='%Y-%m-%d').astype('str')
DF['end'] = pd.to_datetime(DF['end'], format='%Y-%m-%d').astype('str')

In [6]:
DF

,title,start,end,url,description
0,Urban Oasis 輕珠寶x風格選物快閃店,2019-03-14,2019-05-19,https://www.huashan1914.com/w/huashan1914/exhi...,[期間限定店]
1,夏姿‧陳 針間絮語,2019-05-10,2019-05-19,https://www.huashan1914.com/w/huashan1914/exhi...,[展演活動]
2,華山文旅學 X 今秋移住計畫,2019-05-17,2019-05-19,https://www.huashan1914.com/w/huashan1914/exhi...,[市集活動]
3,2019【突破論壇】新時代政府-高教鬆綁,2019-05-25,2019-05-25,https://www.huashan1914.com/w/huashan1914/exhi...,[論壇講座]
4,科創講堂 Tech-Talk FORUM 2019 – 智慧城市,2019-05-26,2019-05-26,https://www.huashan1914.com/w/huashan1914/exhi...,[論壇講座]
5,HANDBUN 手拿包刈包,2019-01-28,2019-05-26,https://www.huashan1914.com/w/huashan1914/exhi...,[期間限定店]
6,2019美麗生活節,2019-05-24,2019-05-27,https://www.huashan1914.com/w/huashan1914/exhi...,[展演活動]
7,2019東方紋繡藝術祭,2019-05-29,2019-05-29,https://www.huashan1914.com/w/huashan1914/exhi...,[展演活動]
8,MASKING奔跑吧！膜王,2019-06-01,2019-06-01,https://www.huashan1914.com/w/huashan1914/exhi...,[品牌活動]
9,日本特選品展MIJ Special Select Shop Fair Summer 2019,2019-05-31,2019-06-02,https://www.huashan1914.com/w/huashan1914/exhi...,[市集活動]
